In [72]:
import pandas as pd

import plotly.express as pe


#model

from sklearn.cluster import KMeans

#metrics

from sklearn.metrics import silhouette_score, calinski_harabasz_score

### I want to market high risk high rewards mutual funds investment plan
### The plan requires heavy investment every year
### the plan is aimed at individuals who have more free cash available and can invest for a lot of years

In [73]:
path = r"C:\Users\harsh\Desktop\NPCI-Python-ML\datasets\defaulters_data.csv"

df = pd.read_csv(path)

df

clientid        income        age         loan  default
0            1  66155.925095  59.017015  8106.532131        0
1            2  34415.153966  48.117153  6564.745018        0
2            3  57317.170063  63.108049  8020.953296        0
3            4  42709.534201  45.751972  6103.642260        0
4            5  66952.688845  18.584336  8770.099235        1
...        ...           ...        ...          ...      ...
1995      1996  59221.044874  48.518179  1926.729397        0
1996      1997  69516.127573  23.162104  3503.176156        0
1997      1998  44311.449262  28.017167  5522.786693        1
1998      1999  43756.056605  63.971796  1622.722598        0
1999      2000  69436.579552  56.152617  7378.833599        0

[2000 rows x 5 columns]

### step 2: Data exploration

In [74]:
df.shape

(2000, 5)

In [75]:
df.index

RangeIndex(start=0, stop=2000, step=1)

In [76]:
df.columns

Index(['clientid', 'income', 'age', 'loan', 'default'], dtype='object')

In [77]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   clientid  2000 non-null   int64  
 1   income    2000 non-null   float64
 2   age       1997 non-null   float64
 3   loan      2000 non-null   float64
 4   default   2000 non-null   int64  
dtypes: float64(3), int64(2)
memory usage: 78.3 KB


In [78]:
df.isna().sum()

clientid    0
income      0
age         3
loan        0
default     0
dtype: int64

In [79]:
df.nunique()

clientid    2000
income      2000
age         1997
loan        2000
default        2
dtype: int64

In [80]:
df["default"].value_counts()

0    1717
1     283
Name: default, dtype: int64

### features

In [81]:
real_value_features = ["loan", "age", "income"]

In [82]:
df[     real_value_features     ].describe()

loan          age        income
count   2000.000000  1997.000000   2000.000000
mean    4444.369695    40.807559  45331.600018
std     3045.410024    13.624469  14326.327119
min        1.377630   -52.423280  20014.489470
25%     1939.708847    28.990415  32796.459717
50%     3974.719419    41.317159  45789.117313
75%     6432.410625    52.587040  57791.281668
max    13766.051239    63.971796  69995.685578

## confirm outliers

In [83]:
for col in real_value_features :
    fig = pe.box(   y=col, data_frame=df  )
    display(fig)

### step 2b) Preprocessing the data!

In [84]:
condition = (df["age"] >= 0)

data = df[    condition    ].copy() #makes new table "data"

del df #delete reference to original table


data.dropna(inplace=True) #drop missing values. DO THIS INPLACE meaning in the same object!

data.reset_index(drop=True, inplace=True)

### cluster this data in such a way that income and age become factors of similarity


# what are the right number of clusters?

In [85]:
features = ["income", "age"]

ans = []

for count in range(2, 21,1):
    model = KMeans(n_clusters=count, n_init="auto")
    model.fit_transform(data[features])
    ans.append(   model.inertia_   )



pe.line(

    x=range(2,21),
    y=ans,
    markers=True
)

### final model

In [86]:
model = KMeans(n_clusters=4, n_init="auto")

#1994
values= model.fit_predict(data[features])

predictions = pd.DataFrame(values, columns=["predicted_cluster"])


result_df = pd.concat(   [data, predictions], axis=1    )

result_df["predicted_cluster"] = result_df["predicted_cluster"].astype("object")

result_df

clientid        income        age         loan  default  \
0            1  66155.925095  59.017015  8106.532131        0   
1            2  34415.153966  48.117153  6564.745018        0   
2            3  57317.170063  63.108049  8020.953296        0   
3            4  42709.534201  45.751972  6103.642260        0   
4            5  66952.688845  18.584336  8770.099235        1   
...        ...           ...        ...          ...      ...   
1989      1996  59221.044874  48.518179  1926.729397        0   
1990      1997  69516.127573  23.162104  3503.176156        0   
1991      1998  44311.449262  28.017167  5522.786693        1   
1992      1999  43756.056605  63.971796  1622.722598        0   
1993      2000  69436.579552  56.152617  7378.833599        0   

     predicted_cluster  
0                    1  
1                    2  
2                    3  
3                    0  
4                    1  
...                ...  
1989                 3  
1990                 1  
1991                 0  
1992                 0  
1993                 1  

[1994 rows x 6 columns]

In [87]:
pe.scatter(
    x="income",
    y="age",
    data_frame=result_df,
    color="predicted_cluster"
)

In [88]:
pe.scatter(
    x="income",
    y="age",
    data_frame=result_df,
    facet_col="predicted_cluster",
)

In [89]:
silhouette_score(result_df[features],   result_df["predicted_cluster"] )

0.5602866133668587